# 02: TimeGAN

The implementation of the TimeGAN is based on the [TimeGAN-tensorflow2-Repository](https://github.com/mcps5601/TimeGAN-tensorflow2)

In [1]:
%load_ext autoreload

%autoreload 2

In [2]:
from synthesizers.timegan.timegan import train_timegan
import numpy as np
import os

In [ ]:
windows = np.load('data/wesad/wesad_windows.npy')
labels = np.load('data/wesad/wesad_labels.npy')

stress_features = windows[labels == 1]
non_stress_features = windows[labels == 0]

stress_features = np.delete(stress_features, 6, axis=2)
non_stress_features = np.delete(non_stress_features, 6, axis=2)

stress_features.shape, non_stress_features.shape

In [8]:
# Inputs for the main function
EXP_NAME = 'timegan_test'
DATA_NAME = 'wesad'
USE_GAIN = False
MAX_SEQ_LEN = 60
IMP_METHOD = 'median'
TRAIN_RATE = 1
FEATURE_PREDICTION_NO = 5
SEED = 0
HIDER_MODEL = 'timegan'
NOISE_SIZE = 0.1
SEEKER_MODEL = 'binary_predictor'
TRAIN_ON_STRESS_DATA = False

# Hider params
GEN_TYPE = 'gan'
MODULE_NAME = 'gru'
EPSILON = 1e-8
OPTIMIZER = 'adam'
USE_DPSGD = False
BATCH_SIZE = 128
Z_DIM = -1
HIDDEN_DIM = 10
NUM_LAYERS = 3
EMBEDDING_ITERATIONS = 2000
SUPERVISED_ITERATIONS = 1000
JOINT_ITERATIONS = 6000
ETA = 0.1
NORMALIZATION = False

# DP params
DP_TRAINING = False
L2_NORM_CLIP = 1.0
NOISE_MULTIPLIER = 0.1
DP_LR = 0.15

# Additional variables
FEATURE_DIM = stress_features.shape[-1]

config = {
    'exp_name': EXP_NAME,
    'data_name': DATA_NAME,
    'use_gain': USE_GAIN,
    'max_seq_len': MAX_SEQ_LEN,
    'imp_method': IMP_METHOD,
    'train_rate': TRAIN_RATE,
    'feature_prediction_no': FEATURE_PREDICTION_NO,
    'seed': SEED,
    'hider_model': HIDER_MODEL,
    'noise_size': NOISE_SIZE,
    'seeker_model': SEEKER_MODEL,
    'gen_type': GEN_TYPE,
    'module_name': MODULE_NAME,
    'epsilon': EPSILON,
    'optimizer': OPTIMIZER,
    'use_dpsgd': USE_DPSGD,
    'batch_size': BATCH_SIZE,
    'z_dim': Z_DIM,
    'hidden_dim': HIDDEN_DIM,
    'num_layers': NUM_LAYERS,
    'embedding_iterations': EMBEDDING_ITERATIONS,
    'supervised_iterations': SUPERVISED_ITERATIONS,
    'joint_iterations': JOINT_ITERATIONS,
    'eta': ETA,
    'l2_norm_clip': L2_NORM_CLIP,
    'noise_multiplier': NOISE_MULTIPLIER,
    'dp_lr': DP_LR,
    'feature_dim': FEATURE_DIM,
    'dp_training': DP_TRAINING,
    'normalization': NORMALIZATION,
    'train_on_stress_data': TRAIN_ON_STRESS_DATA,
}

In [9]:
import random
def data_division (data, seed, divide_rates):
  """Divide the dataset into sub datasets.
  
  Args:
    - data: original data (list format)
    - seed: random seed
    - divide_rates: ratio for each division
    
  Returns:
    - divided_data: divided data (list format)
    - divided_index: divided data index (list format)
  """
  # sum of the division rates should be 1
  assert sum(divide_rates) == 1
  
  # Output initialization
  divided_data = list()
  divided_index = list()
  
  # Set index
  no = len(data)
  random.seed(seed)
  index = np.random.permutation(no)

  # Set divided index & data
  for i in range(len(divide_rates)):
    temp_idx = index[int(no*sum(divide_rates[:i])):int(no*sum(divide_rates[:(i+1)]))]
    divided_index.append(temp_idx)
    
    temp_data = [data[j] for j in temp_idx]
    divided_data.append(temp_data)
  
  return divided_data, divided_index

In [10]:
# Divide the data into training and testing

divided_data, _ = data_division(stress_features if TRAIN_ON_STRESS_DATA else non_stress_features, 
                                seed=SEED, 
                                divide_rates=[TRAIN_RATE, 1-TRAIN_RATE])

train_data = np.asarray(divided_data[0])
test_data = np.asarray(divided_data[1])

train_data.shape, non_stress_features.shape, test_data.shape

Finish data loading: wesad


In [12]:
model, max_val, min_val, train_log_dir = train_timegan(train_data, 0, config)
os.system("say 'TimeGAN IST FERTIG'")

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: nw20hewo. Use `wandb login --relogin` to force relogin


TRAINING WITH FOLLOWING CONFIG:
{'exp_name': 'timegan_test', 'data_name': 'wesad', 'use_gain': False, 'max_seq_len': 60, 'imp_method': 'median', 'train_rate': 1, 'feature_prediction_no': 5, 'seed': 0, 'hider_model': 'timegan', 'noise_size': 0.1, 'seeker_model': 'binary_predictor', 'gen_type': 'gan', 'module_name': 'gru', 'epsilon': 1e-08, 'optimizer': 'adam', 'use_dpsgd': False, 'batch_size': 128, 'z_dim': -1, 'hidden_dim': 10, 'num_layers': 3, 'embedding_iterations': 2000, 'supervised_iterations': 1000, 'joint_iterations': 6000, 'eta': 0.1, 'l2_norm_clip': 1.0, 'noise_multiplier': 0.1, 'dp_lr': 0.15, 'feature_dim': 6, 'dp_training': False, 'normalization': False, 'train_on_stress_data': False}
NO DATA normalized
Start Embedding Network Training
step: 0/2000, e_loss: 0.5501
step: 100/2000, e_loss: 0.2081


KeyboardInterrupt: 

In [12]:
def random_generator(batch_size, z_dim, T_mb, max_seq_len):
    """Random vector generation.
    Args:
        batch_size: size of the random vector
        z_dim: dimension of random vector
        T_mb: time information for the random vector
        max_seq_len: maximum sequence length
    Return:
        Z_mb: generated random vector
    """
    Z_mb = list()
    for i in range(batch_size):
        temp = np.zeros([max_seq_len, z_dim])
        temp_Z = np.random.uniform(0., 1, [T_mb[i], z_dim])
        temp[:T_mb[i],:] = temp_Z
        Z_mb.append(temp_Z)

    return Z_mb


Save non-stress dataset

In [14]:
import tensorflow as tf
# save synthetic data

max_seq_len = MAX_SEQ_LEN
num_syn_samples = 540
ori_time = [60] * num_syn_samples
Z_mb = random_generator(num_syn_samples, config['z_dim'], ori_time, max_seq_len)
Z_mb = tf.convert_to_tensor(Z_mb, dtype=tf.float32)
generated_data = model.generate(Z_mb, num_syn_samples, ori_time, max_val, min_val)
zeros = np.zeros([num_syn_samples, 60, 1])
gen_data = np.append(np.array(generated_data), zeros, axis=2)

with open (f"/Users/nils/thesis/Data_Generation/data/syn/timegan/no_dp/non_stress/syn_dataset_540.npy", "wb") as f:
   np.save(f, gen_data)

Save stress dataset

In [68]:
max_seq_len = MAX_SEQ_LEN
num_syn_samples = 540
ori_time = [60] * num_syn_samples
Z_mb = random_generator(num_syn_samples, config['z_dim'], ori_time, max_seq_len)
Z_mb = tf.convert_to_tensor(Z_mb, dtype=tf.float32)
generated_data = model.generate(Z_mb, num_syn_samples, ori_time, max_val, min_val)
ones = np.ones([num_syn_samples, 60, 1])
gen_data = np.append(np.array(generated_data), ones, axis=2)

with open (f"data/syn/timegan/no_dp/stress/syn_dataset_540.npy", "wb") as f:
   np.save(f, gen_data)

Save non-stress subject data

In [15]:
# save synthetic data
num_syn_samples = 540 // 15
ori_time = [60] * num_syn_samples
Z_mb = random_generator(num_syn_samples, config['z_dim'], ori_time, max_seq_len)
Z_mb = tf.convert_to_tensor(Z_mb, dtype=tf.float32)
generated_data = model.generate(Z_mb, num_syn_samples, ori_time, max_val, min_val)
zeros = np.zeros([num_syn_samples, 60, 1])
gen_data = np.append(np.array(generated_data), zeros, axis=2)

with open (f"data/syn/timegan/no_dp/non_stress/syn_subject_36.npy", "wb") as f:
    np.save(f, gen_data)

Save stress subject data

In [67]:
# save synthetic data
num_syn_samples = 540 // 15
ori_time = [60] * num_syn_samples
Z_mb = random_generator(num_syn_samples, config['z_dim'], ori_time, max_seq_len)
Z_mb = tf.convert_to_tensor(Z_mb, dtype=tf.float32)
generated_data = model.generate(Z_mb, num_syn_samples, ori_time, max_val, min_val)
ones = np.ones([num_syn_samples, 60, 1])
gen_data = np.append(np.array(generated_data), ones, axis=2)

with open (f"data/syn/timegan/no_dp/stress/syn_subject_36.npy", "wb") as f:
    np.save(f, gen_data)